In [1]:
using StatsBase
using GLM
using QuantileRegression
using DataFrames
using Gadfly
using Cairo
using Fontconfig
using Formatting
using Compose, Colors


Use "abstract type Solver end" instead.


In [2]:
# get the data into a DataFrame

fulldf = DataFrame()

probnames = [:foo, :galid, :groupid, :cenid, :rband, :Psat, :Mh, :foo2, :foo3, :foo4, :projR, :projrad, :angRh]
corrnames = [:foo5, :galid, :M_r, :M_g, :cz, :Dn4000, :H_delta, :logsSFR, :stelM, :ra, :dec, :vdisp, :S2N, :sersic,
             :conc, :KplusA, :R_exp, :surfdens1kpc, :surfdensR_e, :surfdensR_eo2, :vdisp_err, :Bulge2Tlr, :B2T_err,
             :GMoR_e, :R_e]

probfiles = ["dat/clf_groups_M18_M9.4.prob", "dat/clf_groups_M19_M9.8.prob", "dat/clf_groups_M20_M10.3.prob"]
corrfiles = ["dat/clf_groups_M18_M9.4.galdata_corr", "dat/clf_groups_M19_M9.8.galdata_corr", "dat/clf_groups_M20_M10.2.galdata_corr"]
densfiles = ["dat/density_r10.M18_M9.4", "dat/density_r10.M19_M9.8", "dat/density_r10.M20_M10.3"]
randfiles = ["dat/drandom_r10.M18_M9.4", "dat/drandom_r10.M19_M9.8", "dat/drandom_r10.M20_M10.3"]

for i in 1:3
    
    # read the two catalogues in and join them on galaxy id
    probdf = readtable(probfiles[i], separator=' ', header=false)
    names!(probdf, probnames)
    corrdf = readtable(corrfiles[i], separator=' ', header=false)
    names!(corrdf, corrnames)
    
    # add a column for the density, corrected with the randoms
    density = readdlm(densfiles[i])
    rands = readdlm(randfiles[i])
    ρ_corr = 1.25 * (density[:, 1] ./ rands[:, 1])
    probdf[:ρ_env] = DataArray(ρ_corr)
    
    joindf = join(probdf, corrdf, on=:galid)
   
    if i == 1
        fulldf = joindf
    else
        fulldf = [fulldf; joindf]
    end
end

fulldf = unique(fulldf, :galid)
    

# remove any galaxies in environment density less than or equal to zero, to avoid computational issues
fulldf = fulldf[fulldf[:ρ_env] .> 0.0, :]

# remove bad measurements of R_e, vdisp (i.e. where they are zero)
fulldf = fulldf[fulldf[:R_e] .> 0.0, :]
fulldf = fulldf[fulldf[:vdisp] .> 0.0, :]


# Take only the quenched galaxies
fulldf = fulldf[fulldf[:Dn4000] .> 1.6, :]

# add a column for log10 of stellar mass and log of environment
fulldf[:log10M] = log10.(fulldf[:stelM])
fulldf[:logρ] = log10.(fulldf[:ρ_env])
fulldf[:logρ2] = fulldf[:logρ] .^ 2
fulldf[:logM2] = fulldf[:log10M] .^ 2

# Now take all the rows out which don't have central galaxies
fulldf = fulldf[fulldf[:Psat] .< 0.5, :]

,foo,galid,groupid,cenid,rband,Psat,Mh,foo2,foo3,foo4,projR,projrad,angRh,ρ_env,foo5,M_r,M_g,cz,Dn4000,H_delta,logsSFR,stelM,ra,dec,vdisp,S2N,sersic,conc,KplusA,R_exp,surfdens1kpc,surfdensR_e,surfdensR_eo2,vdisp_err,Bulge2Tlr,B2T_err,GMoR_e,R_e,log10M,logρ,logρ2,logM2
1,PROB10,17,2485,17,-20.45934,0.0,1.505301e12,0.0,1.505301e12,0.0,0.0,0.0,0.002489693,4.620534098919567,GALDAT_CORR,-20.459339,-19.52249,11982.790039,1.93181,-2.23705,-11.93087,2.32574e10,0.977202,0.007641,224.654999,43.082401,6.0,3.289537,0,4.18451,8.5071e8,2.04648e9,2.46656e9,4.68927,0.71,0.01,107.792,0.002568,10.366561162294122,0.6646921796268715,0.44181569365712126,107.46559033158486
2,PROB10,83,824,83,-20.82109,0.0,4.769398e12,2.052487,4.769398e12,2.052487,0.0,0.0,0.005981735,4.037491605715396,GALDAT_CORR,-20.821091,-19.855761,7303.350098,1.81796,-0.95315,-11.29768,3.41598e10,0.721953,0.015959,195.688004,40.283001,2.68589,3.185269,0,6.62674,3.83006e8,9.00621e8,1.91893e9,4.62724,-999.0,0.0,96.849701,0.004839,10.533515319301785,0.6061116324053456,0.3673713109370728,110.95494498196537
3,PROB10,93,607,93,-21.37808,0.0,6.432387e12,0.0,6.432387e12,0.0,0.0,0.0,0.005765921,3.209461072780544,GALDAT_CORR,-21.37808,-20.40151,8376.849609,1.85702,-1.32328,-12.13293,5.50736e10,0.736818,0.017271,202.039001,42.187801,3.71331,2.855232,0,7.29463,4.53193e8,1.0304e9,2.5889e9,4.49715,-999.0,0.0,118.281998,0.005232,10.740943465968014,0.5064321125396518,0.25647348461137454,115.36786653912098
4,PROB10,99,11864,99,-18.71293,0.0,2.641059e11,0.0,2.641059e11,0.0,0.0,0.0,0.001698125,0.8999116839234984,GALDAT_CORR,-18.712931,-17.87118,9821.610352,1.73772,-0.40409,-11.6214,4.02227e9,0.74412,0.015701,79.491302,28.9055,3.04461,2.688282,0,1.67945,9.26302e8,1.67391e9,8.25183e8,4.07199,0.77,0.07,60.6306,0.000973,9.60447121980981,-0.04580010952366242,0.0020976500323794727,92.24586741215494
5,PROB10,218,1424,218,-20.83903,0.0,2.721754e12,0.0,2.721754e12,0.0,0.0,0.0,0.003302211,4.883816881291285,GALDAT_CORR,-20.839029,-19.84749,10999.320312,1.93528,-2.63873,-12.22909,3.40868e10,0.863652,0.018245,188.425995,33.013401,2.7964,2.796242,0,7.44174,2.08103e8,4.62428e8,8.36261e8,5.50887,0.45,0.0,62.121498,0.003958,10.53258623245291,0.6887593717069869,0.4743894721142033,110.9353727440566
6,PROB10,222,1611,222,-20.31638,0.0,2.39733e12,2.0,2.39733e12,2.0,0.0,0.0,0.003110063,5.020097345572935,GALDAT_CORR,-20.31638,-19.36981,11196.519531,1.88434,-2.14496,-12.04822,2.1236e10,0.869052,0.017126,174.727005,31.5695,6.0,3.009981,0,0.0,3.37969e8,8.7565e8,1.49198e9,5.29266,0.49,0.01,91.769203,0.003496,10.32707271666912,0.7007121387058289,0.49099750132969683,106.64843089537172
7,PROB10,237,12835,237,-18.4337,0.0,2.414243e11,0.0,2.414243e11,0.0,0.0,0.0,0.001629321,1.9192095283615083,GALDAT_CORR,-18.433701,-17.635509,9935.240234,1.80145,3.62374,-10.67433,3.47169e9,0.883338,0.018246,59.784698,11.1745,0.989295,2.479145,0,6.88493,1.19009e8,2.50285e8,3.26235e8,15.2537,0.05,0.02,35.456402,0.003109,9.54054093848024,0.2831223911209624,0.0801582883540512,91.02192139881741
8,PROB10,267,3524,267,-20.21592,0.0,1.026703e12,0.0,1.026703e12,0.0,0.0,0.0,0.002395382,4.00472930438155,GALDAT_CORR,-20.215919,-19.33007,10955.929688,1.84876,-1.70121,-11.75834,1.78245e10,0.928125,0.017808,140.399994,40.9883,4.43531,2.961898,0,3.05284,8.52206e8,1.8651e9,1.742e9,3.72171,0.94,0.0,98.398804,0.002007,10.251017356179636,0.6025731657170086,0.3630944200422175,105.08335683669614
9,PROB10,303,9229,303,-19.19465,0.0,3.50912e11,0.0,3.50912e11,0.0,0.0,0.0,0.001530853,4.587221276310996,GALDAT_CORR,-19.194651,-18.335079,11994.019531,1.83937,0.11096,-11.17117,6.3276e9,0.9833,0.015147,90.880096,23.6038,2.24688,2.755007,0,3.76053,4.0102e8,7.58286e8,7.14623e8,5.86189,0.34,0.02,57.5825,0.001982,9.801239017379089,0.6615496899386126,0.4376479922578745,96.06428627579422
10,PROB10,888,7205,888,-19.43074,0.0,4.636953e11,0.0,4.636953e11,0.0,0.0,0.0,0.002488332,2.348306579396965,GALDAT_CORR,-19.43074,-18.48604,8076.899902,1.95121,-1.92263,-11.71874,8.85471e9,3.681371,

In [3]:
nbins = 5
maxM = maximum(fulldf[:log10M])
minM = minimum(fulldf[:log10M])
Medges = logspace(minM, maxM, nbins + 1)

fullmeans = []
fullmeds = []

# set a new column with mean logM for the colours in our plot
fulldf[:meanlogM] = zeros(nrow(fulldf))
for i in 1:nbins
    if i == nbins
        meanM = mean(fulldf[:log10M][log10(Medges[i]) .<= fulldf[:log10M] .<= log10(Medges[i + 1])])
        medianM = median(fulldf[:log10M][log10(Medges[i]) .<= fulldf[:log10M] .<= log10(Medges[i + 1])])
        fulldf[:meanlogM][log10(Medges[i]) .<= fulldf[:log10M] .<= log10(Medges[i + 1])] = meanM
    else    
        meanM = mean(fulldf[:log10M][log10(Medges[i]) .<= fulldf[:log10M] .< log10(Medges[i + 1])])
        medianM = median(fulldf[:log10M][log10(Medges[i]) .<= fulldf[:log10M] .< log10(Medges[i + 1])])
        fulldf[:meanlogM][log10(Medges[i]) .<= fulldf[:log10M] .< log10(Medges[i + 1])] = meanM
    end
    append!(fullmeans, meanM)
    append!(fullmeds, medianM)
end

In [4]:
# get the density bins to plot in
nrobins = 20
maxfro = maximum(fulldf[:logρ])
minfro = minimum(fulldf[:logρ])
frobins = linspace(minfro, maxfro, nrobins + 1)

fromeans = []
for i in 1:nrobins
    if i == nrobins
        meanro = mean(fulldf[:logρ][frobins[i] .<= fulldf[:logρ] .<= frobins[i + 1]])
    else    
        meanro = mean(fulldf[:logρ][frobins[i] .<= fulldf[:logρ] .< frobins[i + 1]])
    end
    append!(fromeans, meanro)
end

In [ ]:
# Dn4k

for i in 1:nbins

    if i == nbins
        subfull = fulldf[log10(Medges[i]) .<= fulldf[:log10M] .<= log10(Medges[i + 1]), :]
    else
        subfull = fulldf[log10(Medges[i]) .<= fulldf[:log10M] .< log10(Medges[i + 1]), :]
    end

    # first find the linear fit to mass and subtract
    subfull[:cenR_e] = mean_center_logmass(subfull[:R_e], subfull[:log10M])
    subpure[:cenR_e] = mean_center_logmass(subpure[:R_e], subpure[:log10M])
    
    # now get the values for each bin in density
    fmeans = []
    fmins = []
    fmaxs = []
    pmeans = []
    pmins = []
    pmaxs = []
    for j in 1:nrobins
        fmean, ferr = get_errbar(subfull[:cenR_e][frobins[j] .<= subfull[:logρ] .<= frobins[j + 1]])
        pmean, perr = get_errbar(subpure[:cenR_e][probins[j] .<= subpure[:logρ] .<= probins[j + 1]])
        append!(fmeans, fmean)
        append!(fmins, fmean - ferr)
        append!(fmaxs, fmean + ferr)
        append!(pmeans, pmean)
        append!(pmins, pmean - perr)
        append!(pmaxs, pmean + perr)
    end
    
    if i == 2
        println(fmins)
    end
    
    if i == nbins
        subplots[i] = plot(x=fromeans, y=fmeans, ymin=fmins, ymax=fmaxs, Geom.point, Geom.errorbar,
                           Guide.xlabel("log_10(δ)"), Guide.ylabel("Dn4000"),
                           Guide.annotation(compose(context(),
                                                    Compose.text(0, 0.7,
                                                    Formatting.format("logMgal = {1:.2f}", fullmeans[i])))))
        
        subplots[i + nbins] = plot(x=promeans, y=pmeans, ymin=pmins, ymax=pmaxs, Geom.point, Geom.errorbar,
                                   Guide.xlabel("log_10(δ)"), Guide.ylabel(nothing),
                                   Guide.annotation(compose(context(),
                                                    Compose.text(0, 0.7,
                                                    Formatting.format("logMgal = {1:.2f}", puremeans[i])))))
    else
        subplots[i] = plot(x=fromeans, y=fmeans, ymin=fmins, ymax=fmaxs, Geom.point, Geom.errorbar,
                           Guide.xlabel(nothing), Guide.ylabel("Dn4000"),
                           Guide.annotation(compose(context(),
                                                    Compose.text(0, 0.7,
                                                    Formatting.format("logMgal = {1:.2f}", fullmeans[i])))))
        
        subplots[i + nbins] = plot(x=promeans, y=pmeans, ymin=pmins, ymax=pmaxs, Geom.point, Geom.errorbar,
                                   Guide.xlabel(nothing), Guide.ylabel(nothing),
                                   Guide.annotation(compose(context(),
                                                    Compose.text(0, 0.7,
                                                    Formatting.format("logMgal = {1:.2f}", puremeans[i])))))
    end
end
    
fig = hstack(vstack(subplots[1:5]), vstack(subplots[6:10]))
draw(PNG("errbar_Dn4k_v_rho.png", 6inch, 15inch), fig)
display("image/png", read("boxplot_Dn4k_v_rho.png"))